## Testes de valência para o projeto final de IA369Y 2 Semestre 2018

Passos para tratar os dados com valência, testar e escolher um classificador para utilizar no projeto final de IA369Y.

1) Remover espaços duplos, quebras de linha, números e links do dataset e das frases a serem testadas.

2) Remover stopwords e aplicar o stemmer.

3) Treinar os classificadores.

4) Realizar as predições com os classificadores.

5) Avaliar as medidas obtidas com os classificadores.

In [23]:
import sys
sys.path.append('../..')

import csv
import codecs
import copy
import re
from random import shuffle

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from nltk import word_tokenize
import gensim

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import Normalizer, MinMaxScaler
from sklearn.decomposition import TruncatedSVD, LatentDirichletAllocation
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, GaussianNB, ComplementNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, confusion_matrix
from sklearn.externals import joblib

from utils import tokenizer, load_six_emotions, load_3_emotions

%matplotlib inline

np.warnings.filterwarnings('ignore')
np.random.seed(12345)

def highlight_max(data, color='green'):
    attr = f'background-color: {color}; color: white; font-weight: bold;'
    #remove % and cast to float
    data = data.replace('%','', regex=True).astype(float)
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        is_max = data == data.max()
        return [attr if v else '' for v in is_max]
    else:  # from .apply(axis=None)
        is_max = data == data.max().max()
        return pd.DataFrame(np.where(is_max, attr, ''),
                            index=data.index, columns=data.columns)

### Matriz de Resultados

In [24]:
classf = {
    'MultinomialNB': 0,
    'ComplementNB': 0,
    'LogisticRegression': 0,
    'RandomForestClassifier': 0,
    'KNeighborsClassifier': 0,
    'MLPClassifier': 0,
    'LinearSVC': 0,
    'SVC': 0
}

matriz_resultados = {
    'all': {
        'tfidf': copy.deepcopy(classf),
        'tfidf+lsa': copy.deepcopy(classf),
        'tfidf+lda': copy.deepcopy(classf),
        'count': copy.deepcopy(classf),
        'count+lsa': copy.deepcopy(classf),
        'count+lda': copy.deepcopy(classf),
        'tfidf+count+w2c': copy.deepcopy(classf),
    },
    'tweets': {
        'tfidf': copy.deepcopy(classf),
        'tfidf+lsa': copy.deepcopy(classf),
        'tfidf+lda': copy.deepcopy(classf),
        'count': copy.deepcopy(classf),
        'count+lsa': copy.deepcopy(classf),
        'count+lda': copy.deepcopy(classf),
        'tfidf+count+w2c': copy.deepcopy(classf),       
    },
    'titulo_noticias': {
        'tfidf': copy.deepcopy(classf),
        'tfidf+lsa': copy.deepcopy(classf),
        'tfidf+lda': copy.deepcopy(classf),
        'count': copy.deepcopy(classf),
        'count+lsa': copy.deepcopy(classf),
        'count+lda': copy.deepcopy(classf),
        'tfidf+count+w2c': copy.deepcopy(classf),
    }
}

## Datasets

Para validar, serão utilizados dois datasets.

Os dos datasets foram obtidos do site minerando dados.

O primeiro deles tem tweets de política de Minas Gerais com rótulos de valência: positivo, negativo e neutro. Foi feito um tratamento para eliminar tweets repetidos e dessa forma sobraram 3016 tweets.

O segundo contém 2123 títulos de notícias com rótulos de valência: positivo, negativo e neutro.

In [25]:
path = '/home/rdenadai/vagrant/python-dev/sentiment-analysis-2018-president-election/dataset/valencia/'

#Carregando os datasets
def carregar(filename):
    frases = []
    with open(filename, 'r', encoding='utf-8') as h:
        reader = csv.reader(h, delimiter='|')
        for row in reader:
            frase = tokenizer(row[0]).strip()
            valencia = row[1].upper()
            if len(frase) > 5:
                frases.append((valencia, frase))
    return frases

In [26]:
#Carrega os datasets em separado
tweets_mg = carregar(f'{path}tweets_mg_tratados.csv')
shuffle(tweets_mg)
titulo_noticias = carregar(f'{path}titulo_noticias.txt')
shuffle(titulo_noticias)

frases = tweets_mg + titulo_noticias
shuffle(frases)

print(frases[:5])
print('-' * 20)
print(tweets_mg[:5])
print('-' * 20)
print(titulo_noticias[:5])

[('POSITIVO', 'bovesp dispar ultrapass mil pont após datafolh valor econôm'), ('POSITIVO', 'amor human men egoíst mais pur desinteress amor amizad'), ('POSITIVO', 'pmg autor roub pres veícul produt crim recuper'), ('POSITIVO', 'govern min entreg veícul transport escol municípi'), ('NEGATIVO', 'após mort menin dois escol interdit exam indic garot hepatit suspeit águ contamin')]
--------------------
[('NEUTRO', 'josleitepereir tem alter program social lul dilm politic estar min'), ('POSITIVO', 'govern min entreg veícul transport escol municípi'), ('POSITIVO', 'pmg veícul produt roub localiz'), ('POSITIVO', 'detran realiz açõ educ trânsit mês carnaval'), ('NEGATIVO', 'capit govern pôr sigil dar sobr vos pimentel')]
--------------------
[('POSITIVO', 'macei faz campanh contr aids joã secretari municipal distribu cartilh preserv durant fest exam ser realiz capital alagoan'), ('NEGATIVO', 'sistem cantareir registr nov qued águ reservatóri notíc paul'), ('NEGATIVO', 'cad rejeit recurs interpo

In [27]:
def carregar(filename):
    frases = []
    with open(filename, 'r', encoding='utf-8') as h:
        reader = csv.reader(h, delimiter='|')
        for row in reader:
            frase = ' '.join(row[0].strip().splitlines())
            valencia = row[1].upper()
            if len(frase) > 5:
                frases.append((valencia, frase))
    return frases

titulo_noticias2 = carregar(f'{path}titulo_noticias.txt')
only_neutro = []
for sent, new in titulo_noticias2:
    if sent == 'NEUTRO':
        only_neutro.append(new)
with open(f'{path}titulo_noticias_neutro.txt', 'w') as h:
    for k in only_neutro:
        h.write(f'{k}|NEUTRO\n')

In [28]:
#all_datasets
afrases = []
avalencias = []
for valencia, frase in frases:
    afrases.append(frase)
    avalencias.append(valencia)
    
print(afrases[:5])
print(avalencias[:5])
print('-' * 20)

#tweets_mg
atweets_mg = []
aval_tweets_mg = []
for valencia, frase in tweets_mg:
    atweets_mg.append(frase)
    aval_tweets_mg.append(valencia)

print(atweets_mg[:5])
print(aval_tweets_mg[:5])
print('-' * 20)

#titulo_noticias
atitulo_noticias = []
aval_titulo_noticias = []
for valencia, frase in titulo_noticias:
    atitulo_noticias.append(frase)
    aval_titulo_noticias.append(valencia)

print(atitulo_noticias[:5])
print(aval_titulo_noticias[:5])

['bovesp dispar ultrapass mil pont após datafolh valor econôm', 'amor human men egoíst mais pur desinteress amor amizad', 'pmg autor roub pres veícul produt crim recuper', 'govern min entreg veícul transport escol municípi', 'após mort menin dois escol interdit exam indic garot hepatit suspeit águ contamin']
['POSITIVO', 'POSITIVO', 'POSITIVO', 'POSITIVO', 'NEGATIVO']
--------------------
['josleitepereir tem alter program social lul dilm politic estar min', 'govern min entreg veícul transport escol municípi', 'pmg veícul produt roub localiz', 'detran realiz açõ educ trânsit mês carnaval', 'capit govern pôr sigil dar sobr vos pimentel']
['NEUTRO', 'POSITIVO', 'POSITIVO', 'POSITIVO', 'NEGATIVO']
--------------------
['macei faz campanh contr aids joã secretari municipal distribu cartilh preserv durant fest exam ser realiz capital alagoan', 'sistem cantareir registr nov qued águ reservatóri notíc paul', 'cad rejeit recurs interpor anhangu mult mil', 'reaçã merc dilm defend contrat petrob

In [29]:
def run_ml_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    accuracy = np.round(model.score(X_test, y_test) * 100, 2)
    print(f'Modelo   : {model.__class__.__name__}')
    print(f'Acurácia : {accuracy}%')
    print('-' * 20)
    return accuracy

def split_data(X, y):
    test_size = .33
    random_state = 0
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    return {
        'X_train': X_train,
        'y_train': y_train,
        'X_test': X_test,
        'y_test': y_test
    }

## Classificadores

In [30]:
classifiers = (
    MultinomialNB(),
    ComplementNB(),
    LogisticRegression(multi_class='auto', solver='lbfgs'),
    RandomForestClassifier(n_estimators=50, min_samples_split=5, random_state=0),
    KNeighborsClassifier(n_neighbors=8, algorithm='auto'),
    MLPClassifier(hidden_layer_sizes=(100, 25), max_iter=500, random_state=0),
    LinearSVC(max_iter=500),
    SVC(gamma='auto', max_iter=500),
)

## TF-IDF

In [31]:
vec_tfidf = TfidfVectorizer(ngram_range=(1, 2))
X_tfidf = vec_tfidf.fit_transform(afrases)

vec_tfidf_tmg = TfidfVectorizer(ngram_range=(1, 2))
X_tfidf_tmg = vec_tfidf_tmg.fit_transform(atweets_mg)

vec_tfidf_tn = TfidfVectorizer(ngram_range=(1, 2))
X_tfidf_tn = vec_tfidf_tn.fit_transform(atitulo_noticias)

In [32]:
print("\nall_datasets")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_tfidf, avalencias))
        matriz_resultados['all']['tfidf'][classifier.__class__.__name__] = acc
    except:
        pass

print("\ntweets_mg")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_tfidf_tmg, aval_tweets_mg))
        matriz_resultados['tweets']['tfidf'][classifier.__class__.__name__] = acc
    except:
        pass

print("\ntitulo_noticias")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_tfidf_tn, aval_titulo_noticias))
        matriz_resultados['titulo_noticias']['tfidf'][classifier.__class__.__name__] = acc
    except:
        pass      


all_datasets
Modelo   : MultinomialNB
Acurácia : 58.3%
--------------------
Modelo   : ComplementNB
Acurácia : 63.19%
--------------------
Modelo   : LogisticRegression
Acurácia : 62.68%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 62.38%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 58.9%
--------------------
Modelo   : MLPClassifier
Acurácia : 62.33%
--------------------
Modelo   : LinearSVC
Acurácia : 64.65%
--------------------
Modelo   : SVC
Acurácia : 54.01%
--------------------

tweets_mg
Modelo   : MultinomialNB
Acurácia : 64.71%
--------------------
Modelo   : ComplementNB
Acurácia : 62.08%
--------------------
Modelo   : LogisticRegression
Acurácia : 66.73%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 65.93%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 63.8%
--------------------
Modelo   : MLPClassifier
Acurácia : 65.02%
--------------------
Modelo   : LinearSVC
Acurácia : 66.13%
-----------------

## LSA (usando TF-IDF)

In [33]:
#all_datasets
svd = TruncatedSVD(n_components=70, n_iter=50, random_state=0)
normalizer = MinMaxScaler(copy=False)
lsa = make_pipeline(svd, normalizer)
X_svd = lsa.fit_transform(X_tfidf)

print("\nall_datasets")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_svd, avalencias))
        matriz_resultados['all']['tfidf+lsa'][classifier.__class__.__name__] = acc
    except Exception as e:
        print(e)

#tweets_mg
X_svd = lsa.fit_transform(X_tfidf_tmg)

print("\ntweets_mg")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_svd, aval_tweets_mg))
        matriz_resultados['tweets']['tfidf+lsa'][classifier.__class__.__name__] = acc
    except:
        pass


#titulo_noticias
X_svd = lsa.fit_transform(X_tfidf_tn)

print("\ntitulo_noticias")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_svd, aval_titulo_noticias))
        matriz_resultados['titulo_noticias']['tfidf+lsa'][classifier.__class__.__name__] = acc
    except:
        pass


all_datasets
Modelo   : MultinomialNB
Acurácia : 41.57%
--------------------
Modelo   : ComplementNB
Acurácia : 52.51%
--------------------
Modelo   : LogisticRegression
Acurácia : 56.33%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 59.42%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 56.41%
--------------------
Modelo   : MLPClassifier
Acurácia : 58.13%
--------------------
Modelo   : LinearSVC
Acurácia : 57.4%
--------------------
Modelo   : SVC
Acurácia : 41.4%
--------------------

tweets_mg
Modelo   : MultinomialNB
Acurácia : 46.31%
--------------------
Modelo   : ComplementNB
Acurácia : 53.89%
--------------------
Modelo   : LogisticRegression
Acurácia : 67.24%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 64.11%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 62.18%
--------------------
Modelo   : MLPClassifier
Acurácia : 66.33%
--------------------
Modelo   : LinearSVC
Acurácia : 67.54%
----------------

## LDA (usando TF-IDF)

In [34]:
#all_datasets
lda = LatentDirichletAllocation(n_components=70, max_iter=50, random_state=0, n_jobs=5)
normalizer = MinMaxScaler(copy=False)
lda = make_pipeline(lda, normalizer)
X_lda = lda.fit_transform(X_tfidf)

print("\nall_datasets")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_lda, avalencias))
        matriz_resultados['all']['tfidf+lda'][classifier.__class__.__name__] = acc
    except:
        pass


#tweets_mg
X_lda = lda.fit_transform(X_tfidf_tmg)

print("\ntweets_mg")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_lda, aval_tweets_mg))
        matriz_resultados['tweets']['tfidf+lda'][classifier.__class__.__name__] = acc
    except:
        pass


#titulo_noticias
X_lda = lda.fit_transform(X_tfidf_tn)

print("\ntitulo_noticias")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_lda, aval_titulo_noticias))
        matriz_resultados['titulo_noticias']['tfidf+lda'][classifier.__class__.__name__] = acc
    except:
        pass


all_datasets
Modelo   : MultinomialNB
Acurácia : 43.03%
--------------------
Modelo   : ComplementNB
Acurácia : 43.37%
--------------------
Modelo   : LogisticRegression
Acurácia : 44.36%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 45.35%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 41.61%
--------------------
Modelo   : MLPClassifier
Acurácia : 43.97%
--------------------
Modelo   : LinearSVC
Acurácia : 44.1%
--------------------
Modelo   : SVC
Acurácia : 42.43%
--------------------

tweets_mg
Modelo   : MultinomialNB
Acurácia : 53.49%
--------------------
Modelo   : ComplementNB
Acurácia : 49.95%
--------------------
Modelo   : LogisticRegression
Acurácia : 53.29%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 51.77%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 49.24%
--------------------
Modelo   : MLPClassifier
Acurácia : 49.44%
--------------------
Modelo   : LinearSVC
Acurácia : 52.88%
---------------

## Count

In [35]:
#all_datasets
vec_count = CountVectorizer(ngram_range=(1, 2))
X_count = vec_count.fit_transform(afrases)

print("\nall_datasets")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_count, avalencias))
        matriz_resultados['all']['count'][classifier.__class__.__name__] = acc
    except:
        pass

      
#tweets_mg
vec_count = CountVectorizer(ngram_range=(1, 2))
X_count_tmg = vec_count.fit_transform(atweets_mg)

print("\ntweets_mg")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_count_tmg, aval_tweets_mg))
        matriz_resultados['tweets']['count'][classifier.__class__.__name__] = acc
    except:
        pass
      
      
#titulo_noticias
vec_count = CountVectorizer(ngram_range=(1, 2))
X_count_tn = vec_count.fit_transform(atitulo_noticias)

print("\ntitulo_noticias")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_count_tn, aval_titulo_noticias))
        matriz_resultados['titulo_noticias']['count'][classifier.__class__.__name__] = acc
    except:
        pass


all_datasets
Modelo   : MultinomialNB
Acurácia : 62.98%
--------------------
Modelo   : ComplementNB
Acurácia : 63.06%
--------------------
Modelo   : LogisticRegression
Acurácia : 64.39%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 63.23%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 32.52%
--------------------
Modelo   : MLPClassifier
Acurácia : 62.68%
--------------------
Modelo   : LinearSVC
Acurácia : 63.19%
--------------------
Modelo   : SVC
Acurácia : 43.8%
--------------------

tweets_mg
Modelo   : MultinomialNB
Acurácia : 61.98%
--------------------
Modelo   : ComplementNB
Acurácia : 60.57%
--------------------
Modelo   : LogisticRegression
Acurácia : 65.62%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 66.63%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 53.89%
--------------------
Modelo   : MLPClassifier
Acurácia : 65.12%
--------------------
Modelo   : LinearSVC
Acurácia : 65.52%
---------------

## LSA (usando Count)

In [36]:
#all_datasets
svd = TruncatedSVD(n_components=70, n_iter=50, random_state=0)
normalizer = MinMaxScaler(copy=False)
lda = make_pipeline(svd, normalizer)
X_svd = lda.fit_transform(X_count)

print("\nall_datasets")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_svd, avalencias))
        matriz_resultados['all']['count+lsa'][classifier.__class__.__name__] = acc
    except:
        pass
      

#tweets_mg
X_svd = lda.fit_transform(X_count_tmg)

print("\ntweets_mg")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_svd, aval_tweets_mg))
        matriz_resultados['tweets']['count+lsa'][classifier.__class__.__name__] = acc
    except:
        pass
      
      
#titulos_noticias
X_svd = lda.fit_transform(X_count_tn)

print("\ntitulos_noticias")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_svd, aval_titulo_noticias))
        matriz_resultados['titulo_noticias']['count+lsa'][classifier.__class__.__name__] = acc
    except:
        pass


all_datasets
Modelo   : MultinomialNB
Acurácia : 41.53%
--------------------
Modelo   : ComplementNB
Acurácia : 47.58%
--------------------
Modelo   : LogisticRegression
Acurácia : 58.04%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 58.43%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 53.67%
--------------------
Modelo   : MLPClassifier
Acurácia : 58.9%
--------------------
Modelo   : LinearSVC
Acurácia : 57.79%
--------------------
Modelo   : SVC
Acurácia : 43.63%
--------------------

tweets_mg
Modelo   : MultinomialNB
Acurácia : 53.59%
--------------------
Modelo   : ComplementNB
Acurácia : 57.94%
--------------------
Modelo   : LogisticRegression
Acurácia : 64.0%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 65.32%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 64.21%
--------------------
Modelo   : MLPClassifier
Acurácia : 66.33%
--------------------
Modelo   : LinearSVC
Acurácia : 66.23%
----------------

## LDA (usando Count)

In [37]:
#all_datasets
lda = LatentDirichletAllocation(n_components=70, max_iter=50, random_state=0, n_jobs=5)
normalizer = MinMaxScaler(copy=False)
lda = make_pipeline(lda, normalizer)
X_lda = lda.fit_transform(X_count)

print("\nall_datasets")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_lda, avalencias))
        matriz_resultados['all']['count+lda'][classifier.__class__.__name__] = acc
    except:
        pass


#tweets_mg
X_lda = lda.fit_transform(X_count_tmg)

print("\ntweets_mg")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_lda, aval_tweets_mg))
        matriz_resultados['tweets']['count+lda'][classifier.__class__.__name__] = acc
    except:
        pass
      
      
#titulo_noticias
X_lda = lda.fit_transform(X_count_tn)

print("\ntitulo_noticias")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X_lda, aval_titulo_noticias))
        matriz_resultados['titulo_noticias']['count+lda'][classifier.__class__.__name__] = acc
    except:
        pass


all_datasets
Modelo   : MultinomialNB
Acurácia : 45.13%
--------------------
Modelo   : ComplementNB
Acurácia : 43.41%
--------------------
Modelo   : LogisticRegression
Acurácia : 46.16%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 48.22%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 43.11%
--------------------
Modelo   : MLPClassifier
Acurácia : 45.05%
--------------------
Modelo   : LinearSVC
Acurácia : 46.29%
--------------------
Modelo   : SVC
Acurácia : 43.33%
--------------------

tweets_mg
Modelo   : MultinomialNB
Acurácia : 57.63%
--------------------
Modelo   : ComplementNB
Acurácia : 55.11%
--------------------
Modelo   : LogisticRegression
Acurácia : 59.56%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 56.93%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 52.78%
--------------------
Modelo   : MLPClassifier
Acurácia : 56.52%
--------------------
Modelo   : LinearSVC
Acurácia : 58.85%
--------------

## Count + TF-IDF + Word2Vec

In [38]:
#all_datasets
# Count
vec_count = CountVectorizer()
X_count = vec_count.fit_transform(afrases)
weights_count = pd.DataFrame(np.round(X_count.toarray().T, 8), index=vec_count.get_feature_names())

# TF-IDF
vec_tfidf = TfidfVectorizer()
X_tfidf = vec_tfidf.fit_transform(afrases)
weights_tfidf = pd.DataFrame(np.round(X_tfidf.toarray().T, 8), index=vec_tfidf.get_feature_names())

# Word2Vec preprocessing
frases_w2v = []
for frase in afrases:
    bigram = []
    p_frase = word_tokenize(frase)
    for m, palavra in enumerate(p_frase):
        next_p = None
        try:
            next_p = p_frase[m+1]
        except:
            pass
        bigram += [f'{palavra}']
#         if next_p:
#             bigram += [f'{palavra} {next_p}']
    frases_w2v += [bigram]

# Word2Vec
model = gensim.models.Word2Vec(
    sentences=frases_w2v,
    sg=1,
    hs=1,
    size=1,
    window=25,
    min_count=1,
    seed=0,
    workers=10)
model.train(frases_w2v, total_examples=len(frases_w2v), epochs=1000)

(65076275, 70428000)

In [39]:
#all_datasets
r_words = {}
for word in vec_count.get_feature_names():
    idx = weights_count.index.get_loc(word)
    w2c_val = .1
    try:
        w2c_val = model.wv[word]
    except:
        pass
    r_words[word] = (weights_tfidf.iloc[idx].values + weights_count.iloc[idx].values) * w2c_val
lwor = list(r_words.keys())
X = np.asarray(list(r_words.values()))
weights = pd.DataFrame(X, index=lwor)
X = X.T

normalizer = Normalizer(copy=False)
X = normalizer.fit_transform(X)

print("\nall_datasets")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X, avalencias))
        matriz_resultados['all']['tfidf+count+w2c'][classifier.__class__.__name__] = acc
    except:
        pass


all_datasets
Modelo   : LogisticRegression
Acurácia : 62.81%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 63.15%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 60.23%
--------------------
Modelo   : MLPClassifier
Acurácia : 59.16%
--------------------
Modelo   : LinearSVC
Acurácia : 62.98%
--------------------
Modelo   : SVC
Acurácia : 51.05%
--------------------


In [40]:
#tweets_mg
# Count
vec_count = CountVectorizer()
X_count = vec_count.fit_transform(atweets_mg)
weights_count = pd.DataFrame(np.round(X_count.toarray().T, 8), index=vec_count.get_feature_names())

# TF-IDF
vec_tfidf = TfidfVectorizer()
X_tfidf = vec_tfidf.fit_transform(atweets_mg)
weights_tfidf = pd.DataFrame(np.round(X_tfidf.toarray().T, 8), index=vec_tfidf.get_feature_names())

# Word2Vec preprocessing
frases_w2v = []
for frase in atweets_mg:
    bigram = []
    p_frase = word_tokenize(frase)
    for m, palavra in enumerate(p_frase):
        next_p = None
        try:
            next_p = p_frase[m+1]
        except:
            pass
        bigram += [f'{palavra}']
#         if next_p:
#             bigram += [f'{palavra} {next_p}']
    frases_w2v += [bigram]

# Word2Vec
model = gensim.models.Word2Vec(
    sentences=frases_w2v,
    sg=1,
    hs=1,
    size=1,
    window=25,
    min_count=1,
    seed=0,
    workers=10)
model.train(frases_w2v, total_examples=len(frases_w2v), epochs=1000)

(21303325, 26219000)

In [41]:
#tweets_mg
r_words = {}
for word in vec_count.get_feature_names():
    idx = weights_count.index.get_loc(word)
    w2c_val = .1
    try:
        w2c_val = model.wv[word]
    except:
        pass
    r_words[word] = (weights_tfidf.iloc[idx].values + weights_count.iloc[idx].values) * w2c_val
lwor = list(r_words.keys())
X = np.asarray(list(r_words.values()))
weights = pd.DataFrame(X, index=lwor)
X = X.T

normalizer = Normalizer(copy=False)
X = normalizer.fit_transform(X)

print("\ntweets_mg")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X, aval_tweets_mg))
        matriz_resultados['tweets']['tfidf+count+w2c'][classifier.__class__.__name__] = acc
    except:
        pass


tweets_mg
Modelo   : LogisticRegression
Acurácia : 66.23%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 66.23%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 62.59%
--------------------
Modelo   : MLPClassifier
Acurácia : 61.27%
--------------------
Modelo   : LinearSVC
Acurácia : 65.22%
--------------------
Modelo   : SVC
Acurácia : 64.31%
--------------------


In [42]:
#titulo_noticias
# Count
vec_count = CountVectorizer()
X_count = vec_count.fit_transform(atitulo_noticias)
weights_count = pd.DataFrame(np.round(X_count.toarray().T, 8), index=vec_count.get_feature_names())

# TF-IDF
vec_tfidf = TfidfVectorizer()
X_tfidf = vec_tfidf.fit_transform(atitulo_noticias)
weights_tfidf = pd.DataFrame(np.round(X_tfidf.toarray().T, 8), index=vec_tfidf.get_feature_names())

# Word2Vec preprocessing
frases_w2v = []
for frase in atitulo_noticias:
    bigram = []
    p_frase = word_tokenize(frase)
    for m, palavra in enumerate(p_frase):
        next_p = None
        try:
            next_p = p_frase[m+1]
        except:
            pass
        bigram += [f'{palavra}']
#         if next_p:
#             bigram += [f'{palavra} {next_p}']
    frases_w2v += [bigram]

# Word2Vec
model = gensim.models.Word2Vec(
    sentences=frases_w2v,
    sg=1,
    hs=1,
    size=1,
    window=25,
    min_count=1,
    seed=0,
    workers=10)
model.train(frases_w2v, total_examples=len(frases_w2v), epochs=1000)

(41547220, 44209000)

In [43]:
#titulo_noticias
r_words = {}
for word in vec_count.get_feature_names():
    idx = weights_count.index.get_loc(word)
    w2c_val = .1
    try:
        w2c_val = model.wjv[word]
    except:
        pass
    r_words[word] = (weights_tfidf.iloc[idx].values + weights_count.iloc[idx].values) * w2c_val
lwor = list(r_words.keys())
X = np.asarray(list(r_words.values()))
weights = pd.DataFrame(X, index=lwor)
X = X.T

normalizer = Normalizer(copy=False)
X = normalizer.fit_transform(X)

print("\ntitulo_noticias")
for classifier in classifiers:
    try:
        acc = run_ml_model(classifier, **split_data(X, aval_titulo_noticias))
        matriz_resultados['titulo_noticias']['tfidf+count+w2c'][classifier.__class__.__name__] = acc
    except:
        pass


titulo_noticias
Modelo   : MultinomialNB
Acurácia : 65.57%
--------------------
Modelo   : ComplementNB
Acurácia : 65.2%
--------------------
Modelo   : LogisticRegression
Acurácia : 67.36%
--------------------
Modelo   : RandomForestClassifier
Acurácia : 64.46%
--------------------
Modelo   : KNeighborsClassifier
Acurácia : 61.4%
--------------------
Modelo   : MLPClassifier
Acurácia : 65.42%
--------------------
Modelo   : LinearSVC
Acurácia : 65.87%
--------------------
Modelo   : SVC
Acurácia : 48.44%
--------------------


### Resultado dos Classificadores

#### Resultado dos Classificadores para todas as frases

In [44]:
df = pd.DataFrame.from_dict(matriz_resultados['all'])
df.style.apply(highlight_max).format({
    'tfidf': '{:,.2f}%'.format,
    'tfidf+lsa': '{:,.2f}%'.format,
    'tfidf+lda': '{:,.2f}%'.format,
    'count': '{:,.2f}%'.format,
    'count+lsa': '{:,.2f}%'.format,
    'count+lda': '{:,.2f}%'.format,
    'tfidf+count+w2c': '{:,.2f}%'.format,
})

,tfidf,tfidf+lsa,tfidf+lda,count,count+lsa,count+lda,tfidf+count+w2c
ComplementNB,63.19%,52.51%,43.37%,63.06%,47.58%,43.41%,0.00%
KNeighborsClassifier,58.90%,56.41%,41.61%,32.52%,53.67%,43.11%,60.23%
LinearSVC,64.65%,57.40%,44.10%,63.19%,57.79%,46.29%,62.98%
LogisticRegression,62.68%,56.33%,44.36%,64.39%,58.04%,46.16%,62.81%
MLPClassifier,62.33%,58.13%,43.97%,62.68%,58.90%,45.05%,59.16%
MultinomialNB,58.30%,41.57%,43.03%,62.98%,41.53%,45.13%,0.00%
RandomForestClassifier,62.38%,59.42%,45.35%,63.23%,58.43%,48.22%,63.15%
SVC,54.01%,41.40%,42.43%,43.80%,43.63%,43.33%,51.05%


#### Resultado dos Classificadores para os Tweets MG

In [45]:
df = pd.DataFrame.from_dict(matriz_resultados['tweets'])
df.style.apply(highlight_max).format({
    'tfidf': '{:,.2f}%'.format,
    'tfidf+lsa': '{:,.2f}%'.format,
    'tfidf+lda': '{:,.2f}%'.format,
    'count': '{:,.2f}%'.format,
    'count+lsa': '{:,.2f}%'.format,
    'count+lda': '{:,.2f}%'.format,
    'tfidf+count+w2c': '{:,.2f}%'.format,
})

,tfidf,tfidf+lsa,tfidf+lda,count,count+lsa,count+lda,tfidf+count+w2c
ComplementNB,62.08%,53.89%,49.95%,60.57%,57.94%,55.11%,0.00%
KNeighborsClassifier,63.80%,62.18%,49.24%,53.89%,64.21%,52.78%,62.59%
LinearSVC,66.13%,67.54%,52.88%,65.52%,66.23%,58.85%,65.22%
LogisticRegression,66.73%,67.24%,53.29%,65.62%,64.00%,59.56%,66.23%
MLPClassifier,65.02%,66.33%,49.44%,65.12%,66.33%,56.52%,61.27%
MultinomialNB,64.71%,46.31%,53.49%,61.98%,53.59%,57.63%,0.00%
RandomForestClassifier,65.93%,64.11%,51.77%,66.63%,65.32%,56.93%,66.23%
SVC,62.89%,47.22%,46.41%,46.11%,54.80%,45.40%,64.31%


#### Resultados dos Classificadores para os Títulos de Notícias

In [46]:
df = pd.DataFrame.from_dict(matriz_resultados['titulo_noticias'])
df.style.apply(highlight_max).format({
    'tfidf': '{:,.2f}%'.format,
    'tfidf+lsa': '{:,.2f}%'.format,
    'tfidf+lda': '{:,.2f}%'.format,
    'count': '{:,.2f}%'.format,
    'count+lsa': '{:,.2f}%'.format,
    'count+lda': '{:,.2f}%'.format,
    'tfidf+count+w2c': '{:,.2f}%'.format,
})

,tfidf,tfidf+lsa,tfidf+lda,count,count+lsa,count+lda,tfidf+count+w2c
ComplementNB,66.47%,57.75%,41.43%,60.88%,47.09%,43.07%,65.20%
KNeighborsClassifier,62.74%,55.66%,43.89%,49.70%,56.04%,46.35%,61.40%
LinearSVC,69.00%,64.61%,46.72%,67.36%,63.04%,47.62%,65.87%
LogisticRegression,66.77%,64.61%,46.50%,68.55%,63.19%,47.84%,67.36%
MLPClassifier,66.32%,62.44%,46.42%,66.24%,62.30%,45.60%,65.42%
MultinomialNB,65.28%,47.09%,46.50%,65.87%,46.65%,46.05%,65.57%
RandomForestClassifier,63.56%,62.37%,48.44%,63.34%,60.58%,46.57%,64.46%
SVC,52.76%,44.49%,43.00%,45.60%,44.04%,44.93%,48.44%


### Modelo escolhido e salvo

In [47]:
vec_tfidf = TfidfVectorizer(ngram_range=(1, 2))
X_tfidf = vec_tfidf.fit_transform(afrases)

svd = TruncatedSVD(n_components=70, n_iter=50, random_state=0)
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)
X_svd = lsa.fit_transform(X_tfidf)

svm = LinearSVC(max_iter=1200)
model = CalibratedClassifierCV(svm) 
model.fit(X_svd, avalencias)

accuracy = np.round(model.score(X_svd, avalencias) * 100, 2)
print(f'Modelo   : {model.__class__.__name__}')
print(f'Acurácia : {accuracy}%')


filename = 'tfidf_valence.sav'
joblib.dump(vec_tfidf, filename)

filename = 'lsa_valence.sav'
joblib.dump(lsa, filename)

filename = 'model_valence.sav'
joblib.dump(model, filename)

Modelo   : CalibratedClassifierCV
Acurácia : 60.03%


['model_valence.sav']

In [48]:
model.classes_

array(['NEGATIVO', 'NEUTRO', 'POSITIVO'], dtype='<U8')

In [49]:
y = model.predict_proba(X_svd)
list(np.round(y[0] * 100, 2))

[29.88, 8.4, 61.72]